In [12]:
!pip install dccp


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [7]:
import itertools
import numpy as np

# -------------------------------------------------------
# 1. Define basic parameters
# -------------------------------------------------------
U = 2  # number of users
P_total = 10.0
sigma2 = 1.0

# Channel gains h[i][k]: from user k to receiver i
h = np.array([[1.0, 0.0],
              [0.0, 1.0]]) 
# h[i,k] is |h_{i+1,k+1}| in 1-based math notation.

# -------------------------------------------------------
# 2. Generate all possible "decoding sets" for each receiver
# -------------------------------------------------------
all_subcomps = [(1,1), (1,2), (2,1), (2,2)]

def valid_decoding_sets(U=2):
    """
    Return all possible (D1, D2), where D1 is the set for receiver 1,
    and D2 is the set for receiver 2, with the constraints:
      (1,1),(1,2) in D1, (2,1),(2,2) in D2.
      Possibly includes cross terms.
    """
    # Potential cross subcomps for Rx1 are (2,1),(2,2).
    # Potential cross subcomps for Rx2 are (1,1),(1,2).

    D1_options = []
    cross_1 = [(2,1),(2,2)]
    for r in range(len(cross_1)+1):
        for subset in itertools.combinations(cross_1, r):
            D1 = set([(1,1),(1,2)]) | set(subset)
            D1_options.append(D1)

    D2_options = []
    cross_2 = [(1,1),(1,2)]
    for r in range(len(cross_2)+1):
        for subset in itertools.combinations(cross_2, r):
            D2 = set([(2,1),(2,2)]) | set(subset)
            D2_options.append(D2)

    for D1 in D1_options:
        for D2 in D2_options:
            yield (D1, D2)

# -------------------------------------------------------
# 3. Permutations of a given decoding set
# -------------------------------------------------------
def all_orders(subcomp_set):
    """
    Return all permutations of a given set of subcomponents.
    """
    sc_list = list(subcomp_set)
    return itertools.permutations(sc_list)

# -------------------------------------------------------
# 4. Computing the SINR-based rates
# -------------------------------------------------------
def decode_step(kj, D, order):
    """
    Return the index (0-based) at which subcomponent (k,j) is decoded
    in this receiver's order. If not in D, return None.
    """
    (k, j) = kj
    if kj not in D:
        return None
    return order.index(kj)  # 0-based

def interference_power(i, kj, D, order, p, h, sigma2):
    """
    Compute the interference power at receiver i for subcomponent (k,j)
    at the moment it is decoded (according to the given order).
    """
    (k, j) = kj
    step = decode_step(kj, D, order)
    if step is None:
        # Not decoded by this receiver => no well-defined interference
        return None

    # Subcomponents not decoded yet = all_subcomps minus those in order[:step]
    not_decoded = set(all_subcomps) - set(order[:step])
    Ipow = 0.0
    for (m, n) in not_decoded:
        if (m, n) != (k, j):  # the target signal is not interference
            Ipow += abs(h[i-1, m-1])**2 * p[(m, n)]
    return Ipow

def compute_rates(D1, order1, D2, order2, p, h, sigma2):
    """
    Given decoding sets/orders for Rx1 & Rx2 and power dict p,
    compute feasible rates R[(k,j)] for each subcomponent.
    """
    R = {}
    for (k, j) in all_subcomps:
        # Identify which receivers decode (k,j).
        decode_receivers = []
        if (k, j) in D1:
            decode_receivers.append(1)
        if (k, j) in D2:
            decode_receivers.append(2)

        # If nobody decodes it, rate = 0 (it won't contribute to user k's sum).
        if not decode_receivers:
            R[(k,j)] = 0.0
            continue

        feasible_rates_i = []
        for i_rx in decode_receivers:
            if i_rx == 1:
                Ipow = interference_power(1, (k,j), D1, order1, p, h, sigma2)
                if Ipow is None:
                    # Should not happen if we said receiver 1 decodes it
                    feasible_rates_i.append(0.0)
                    continue
                signal_pow = abs(h[0, k-1])**2 * p[(k, j)]
            else:
                Ipow = interference_power(2, (k,j), D2, order2, p, h, sigma2)
                if Ipow is None:
                    feasible_rates_i.append(0.0)
                    continue
                signal_pow = abs(h[1, k-1])**2 * p[(k, j)]

            sinr = signal_pow / (sigma2 + Ipow)
            feasible_rate = np.log2(1.0 + sinr)
            feasible_rates_i.append(feasible_rate)

        # Overall subcomponent (k,j) rate is limited by the min across decoders
        R[(k,j)] = min(feasible_rates_i)

    return R

# -------------------------------------------------------
# 5. Main enumeration loop (coarse brute force for U=2)
# -------------------------------------------------------
rate_points = []
for (D1, D2) in valid_decoding_sets(U=2):
    for order1 in all_orders(D1):
        for order2 in all_orders(D2):
            # Discretize possible power allocations (p11, p12, p21, p22)
            power_levels = [0.0, 2.0, 4.0, 6.0, 8.0, 10.0]
            for p11 in power_levels:
                for p12 in power_levels:
                    for p21 in power_levels:
                        sum_used = p11 + p12 + p21
                        if sum_used <= P_total:
                            # p22 must keep total <= P_total
                            for p22 in (lvl for lvl in power_levels if (sum_used + lvl) <= P_total):
                                p_dict = {
                                    (1,1): p11,
                                    (1,2): p12,
                                    (2,1): p21,
                                    (2,2): p22
                                }
                                R_sub = compute_rates(D1, order1, D2, order2, p_dict, h, sigma2)
                                # sum up user rates
                                R1 = R_sub[(1,1)] + R_sub[(1,2)]
                                R2 = R_sub[(2,1)] + R_sub[(2,2)]
                                rate_points.append(
                                    (R1, R2, p_dict, D1, D2, order1, order2)
                                )

# -------------------------------------------------------
# 6. Show some top results by sum-rate
# -------------------------------------------------------
rate_points_sorted = sorted(rate_points, key=lambda x: x[0] + x[1], reverse=True)
for i, rp in enumerate(rate_points_sorted[:10]):
    R1, R2, p_dict, D1, D2, ord1, ord2 = rp
    print(f"#{i}: sum_rate={R1+R2:.2f}  R1={R1:.2f}, R2={R2:.2f}, p={p_dict}")
    print(f"     D1={D1}, order1={ord1}")
    print(f"     D2={D2}, order2={ord2}")
    print("---")

#0: sum_rate=5.13  R1=2.32, R2=2.81, p={(1, 1): 0.0, (1, 2): 4.0, (2, 1): 0.0, (2, 2): 6.0}
     D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2))
     D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))
---
#1: sum_rate=5.13  R1=2.32, R2=2.81, p={(1, 1): 0.0, (1, 2): 4.0, (2, 1): 6.0, (2, 2): 0.0}
     D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2))
     D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))
---
#2: sum_rate=5.13  R1=2.81, R2=2.32, p={(1, 1): 0.0, (1, 2): 6.0, (2, 1): 0.0, (2, 2): 4.0}
     D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2))
     D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))
---
#3: sum_rate=5.13  R1=2.81, R2=2.32, p={(1, 1): 0.0, (1, 2): 6.0, (2, 1): 2.0, (2, 2): 2.0}
     D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2))
     D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))
---
#4: sum_rate=5.13  R1=2.81, R2=2.32, p={(1, 1): 0.0, (1, 2): 6.0, (2, 1): 4.0, (2, 2): 0.0}
     D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2))
     D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))
---
#5: sum_rate=5.13  R

In [11]:
import itertools
import numpy as np
import cvxpy as cp

# If you have installed dccp (pip install dccp), you can import it:
import dccp

########################################################################
# Problem Setup
########################################################################

U = 2  # number of users
sigma2 = 1.0

# Channel gains (real or absolute values) h[i][k]: from user k -> receiver i.
# We'll keep it small for demonstration.
h = np.array([[1.0, 0.0],
              [0.0, 1.0]]) 
# h[0, 0] = channel gain from user 1 to Rx1, etc.

# Number of "sub-user" components per user is also U=2 => total subcomps = 4
# We'll label subcomps as (1,1), (1,2), (2,1), (2,2).
all_subcomps = [(1,1), (1,2), (2,1), (2,2)]

# Each user k wants a minimum total rate:
R_min = [0.5, 0.5]   # R1^min=2, R2^min=1

########################################################################
# Decoding subsets (D1, D2)
#   For user i's receiver, it must decode (i,1)...(i,U).
#   It may optionally decode other user's subcomps to remove them via SIC.
########################################################################

def valid_decoding_sets():
    """
    Generate all possible decoding-subset pairs (D1, D2) for U=2.
    Rx1 must decode (1,1),(1,2).
    Rx2 must decode (2,1),(2,2).
    """
    D1_options = []
    cross_1 = [(2,1), (2,2)]  # possible subcomps from user2 that Rx1 might decode
    for r in range(len(cross_1)+1):
        for subset in itertools.combinations(cross_1, r):
            D1 = set([(1,1),(1,2)]) | set(subset)
            D1_options.append(D1)

    D2_options = []
    cross_2 = [(1,1), (1,2)]
    for r in range(len(cross_2)+1):
        for subset in itertools.combinations(cross_2, r):
            D2 = set([(2,1),(2,2)]) | set(subset)
            D2_options.append(D2)

    for D1 in D1_options:
        for D2 in D2_options:
            yield (D1, D2)

########################################################################
# All permutations for a chosen set (decoding order)
########################################################################

def all_orders(subset):
    # Return all permutations (tuples) of the given subset
    return itertools.permutations(list(subset))

########################################################################
# Build the interference sets given the order:
#   If subcomp x is decoded at step t, then all subcomps that appear
#   after t in the order (plus those not in D_i at all) remain as interference.
########################################################################

def decode_step(kj, order):
    """
    Return the index (0-based) at which subcomponent kj is decoded in 'order',
    or None if not in order.
    """
    if kj not in order:
        return None
    return order.index(kj)

def get_interference_set(kj, order, D):
    """
    The set of subcomponents that remain as interference at the moment
    we decode subcomponent 'kj' in the given 'order' for a certain Rx.
    This includes subcomps not in D (never decoded) plus subcomps that
    appear later in the order.
    """
    step = decode_step(kj, order)
    if step is None:
        # Means we're not decoding kj at all at this Rx => no definition.
        return None

    # subcomps not yet decoded: everything not in order[:step]
    already_decoded = set(order[:step])  # subcomps decoded up to step-1
    not_decoded = set(all_subcomps) - already_decoded
    return not_decoded

########################################################################
# We'll define a function that, given a decoding scenario (D1, order1, D2, order2),
# sets up a DCCP problem to:
#  1) introduce variables p_{k,j} >= 0 and R_{k,j} >= 0
#  2) impose rate constraints R_{k,j} <= log2(1 + SNR) for i in decode set
#  3) user sum-of-rates >= R_min
#  4) objective: minimize sum of powers
########################################################################

def solve_for_order(D1, order1, D2, order2, R_min, h, sigma2):
    """
    Returns (status, sum_power_value, p_sol, R_sol)
    or (status='infeasible', None, None, None) if no solution found.

    We'll create the following CVXPY variables:
      p[k,j] >= 0   # power
      R[k,j] >= 0   # rate
    Also define auxiliary variables for the ratio or bilinear constraints needed
    by the "log(1 + x/y)" type expressions in DCCP.
    """
    # 1) Create variables.
    p_vars = {}
    R_vars = {}
    for (k, j) in all_subcomps:
        p_vars[(k,j)] = cp.Variable(nonneg=True, name=f"p_{k}{j}")
        R_vars[(k,j)] = cp.Variable(nonneg=True, name=f"R_{k}{j}")

    # 2) Build constraints list.
    constraints = []

    # 2a) For each subcomponent (k,j), we impose that:
    #     R_{k,j} <= log2(1 + SNR_i) for each receiver i that decodes (k,j).
    #     SNR_i = [p_{k,j} * h[i-1, k-1]^2] / [sigma2 + sum_{(m,n) in Interference} p_{m,n} * h[i-1,m-1]^2].
    #     We'll do that by introducing an auxiliary variable z_{k,j}^{(i)} for the ratio,
    #     and then using the difference-of-convex constraints in DCCP form:
    #        z_{k,j}^{(i)} * (sigma2 + interference) = p_{k,j} * h[i-1,k-1]^2
    #        R_{k,j} <= log2(1 + z_{k,j}^{(i)})
    #
    #     Finally, the actual R_{k,j} is limited by the *minimum* across all decoders i.
    #     Implementation trick: we add multiple constraints:
    #         R_{k,j} <= log2(1 + z_{k,j}^{(i)})   for each i in decode set
    #     so effectively R_{k,j} can't exceed any of them.

    def h_sqr(i, k):
        return (h[i-1, k-1])**2

    # For each receiver i in {1,2}:
    #   if (k,j) in D_i, that means subcomp is decoded by receiver i
    #   => we define the interference set based on the order i.

    # We'll define a small helper to create constraints for subcomp (k,j) at Rx i.
    def add_decoding_constraints(kj, i, order, D):
        # The subcomp kj is decoded at step step_ij:
        step_ij = decode_step(kj, order)
        if step_ij is None:
            return []  # not decoded => no constraints from this Rx
        # Interference set:
        I_set = get_interference_set(kj, order, D)
        if I_set is None:
            return []
        # Build expression for sigma^2 + sum of interfering powers
        I_expr = sigma2
        for (m, n) in I_set:
            if (m, n) != kj:  # the target subcomp is not interference
                I_expr += p_vars[(m,n)] * h_sqr(i, m)

        # We want: z_{kj}^{(i)} >= 0, and
        #          z_{kj}^{(i)} * I_expr = p_{kj} * h_sqr(i, k).
        z = cp.Variable(nonneg=True, name=f"z_{kj}_{i}")
        eq_constraint = (z * I_expr == p_vars[kj] * h_sqr(i, k))

        # Then R_{k,j} <= log2(1 + z) => R_{k,j} <= log(1 + z)/log(2).
        # We'll do: R_vars[kj] <= cp.log(1 + z)/cp.log(2).
        dec_constraint = (R_vars[kj] <= cp.log(1 + z)/cp.log(2))

        return [eq_constraint, dec_constraint]

    # Collect constraints from each subcomp:
    for (k, j) in all_subcomps:
        # If subcomp (k,j) is decoded by Rx1:
        if (k,j) in D1:
            constraints += add_decoding_constraints((k,j), 1, order1, D1)
        # If subcomp (k,j) is decoded by Rx2:
        if (k,j) in D2:
            constraints += add_decoding_constraints((k,j), 2, order2, D2)

        # If subcomp (k,j) is NOT decoded by a receiver, there's no direct constraint
        # bounding R_{k,j}, but it won't help raise user k's rate either.
        # In practice, you'd set R_{k,j} = 0 if no receiver decodes it,
        # but we can let the solver figure that out by the log constraints (none).
        # We'll just keep it feasible: R_{k,j} can be up to something. It's effectively unconstrained from those decoders.

    # 2b) Each user's total rate >= R_min[k].
    # user k's total rate = sum_{j} R_{k,j}.
    for k_user in [1, 2]:
        # sum of R_{k_user, j=1..U}
        user_rate_expr = R_vars[(k_user,1)] + R_vars[(k_user,2)]
        constraints += [user_rate_expr >= R_min[k_user-1]]

    # 3) Objective = minimize sum of powers p_{k,j}.
    objective = cp.Minimize(cp.sum([p_vars[kj] for kj in all_subcomps]))

    # 4) Build and solve the problem with DCCP
    problem = cp.Problem(objective, constraints)

    # Because we have bilinear equality constraints and log(1+z), this is not a
    # standard convex problem. We'll try DCCP's solve (which attempts local solutions).
    try:
        # Use a small maximum number of iterations for demonstration.
        result = problem.solve(method='dccp',
                               solver=cp.MOSEK,  # or another solver if available
                               dc_decomposition=True,
                               max_iter=50,  # you can adjust
                               eps=1e-5)
    except Exception as e:
        print(f"Solver failed on (D1={D1}, order1={order1}, D2={D2}, order2={order2}) with error: {e}")
        return ("error", None, None, None)

    # Check the status
    if problem.status not in ["Converged", "optimal", "Optimal"]:
        return ("infeasible", None, None, None)

    sum_power_val = sum(p_vars[kj].value for kj in all_subcomps)
    # Build dict of power and rates
    p_sol = {kj: p_vars[kj].value for kj in all_subcomps}
    R_sol = {kj: R_vars[kj].value for kj in all_subcomps}
    return (problem.status, sum_power_val, p_sol, R_sol)

########################################################################
# Enumerate decoding sets and orders, pick the best feasible solution
########################################################################

best_sum_power = None
best_solution = None  # will store (D1, order1, D2, order2, p_sol, R_sol)

for (D1, D2) in valid_decoding_sets():
    for order1 in all_orders(D1):
        for order2 in all_orders(D2):

            status, sum_p, p_sol, R_sol = solve_for_order(D1, order1, D2, order2,
                                                          R_min, h, sigma2)
            if status in ["Converged", "optimal", "Optimal"]:
                # Feasible solution found
                if best_sum_power is None or sum_p < best_sum_power:
                    best_sum_power = sum_p
                    best_solution = (D1, order1, D2, order2, p_sol, R_sol)

########################################################################
# Report the result
########################################################################

if best_solution is None:
    print("No feasible solution found that meets the rate requirements.")
else:
    D1, order1, D2, order2, p_sol, R_sol = best_solution
    # Compute final user rates
    R1 = R_sol[(1,1)] + R_sol[(1,2)]
    R2 = R_sol[(2,1)] + R_sol[(2,2)]
    print("===== BEST FEASIBLE SOLUTION =====")
    print(f"Minimum sum power = {best_sum_power:.4f}")
    print(f"Decoding set Rx1 = {D1}, order1 = {order1}")
    print(f"Decoding set Rx2 = {D2}, order2 = {order2}")
    print("Power allocations:")
    for kj in sorted(p_sol.keys()):
        print(f"  p({kj}) = {p_sol[kj]:.4f}")
    print("Rates achieved:")
    for kj in sorted(R_sol.keys()):
        print(f"  R({kj}) = {R_sol[kj]:.4f}")
    print(f"User 1 total rate = {R1:.4f}, User 2 total rate = {R2:.4f}")

Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2)), D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2)), D2={(2, 1), (2, 2)}, order2=((2, 2), (2, 1))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 2), (1, 1)), D2={(2, 1), (2, 2)}, order2=((2, 1), (2, 2))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 2), (1, 1)), D2={(2, 1), (2, 2)}, order2=((2, 2), (2, 1))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2)), D2={(1, 1), (2, 1), (2, 2)}, order2=((1, 1), (2, 1), (2, 2))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2)), D2={(1, 1), (2, 1), (2, 2)}, order2=((1, 1), (2, 2), (2, 1))) with error: Problem is not DCCP.
Solver failed on (D1={(1, 1), (1, 2)}, order1=((1, 1), (1, 2)), D2={(1, 1), (2, 1), (2, 2)}, order2=((2, 1), (

In [ ]:
# Calculate the sub-user component sum
sub_user_component_sum = {k: R_sub[(k, 1)] + R_sub[(k, 2)] for k in range(1, U + 1)}

# Print the sub-user component sum
print(sub_user_component_sum)